# LIMPIEZA DE DATOS 

## Indice notebook 

+ 1.	Analizamos columnas
+ 2.	Limpieza en particular de la columna “place_with_parent_names”
+ 3.	Regex a columna "description”
+ 4.    Outliers Superficie de superficie y precios en USD
+ 5.	Regex a la columna "title"
+ 6.	Resultados Regex de las columnas "description" y "title"
+ 7.	Visualizamos y agregamos los datos obtenidos en “description” y “title”
+ 8.	Conclusiones sobre los Regex y Fillna
+ 9.    Nuevo Dataset limpio 


In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data_location = "C:\DATA SCIENCE\CLASES\desafio 1\properatti.csv"
data = pd.read_csv(data_location)


# 1. Analizamos columnas

### Al analizar las columnas procedemos a ver que es lo que sirve y que cosas no nos son utiles para nuestro objetivo: 


+ INDEX 

si bien **UNNAMED: 0** Indica el índice de la base de datos de Properatti,  nos vamos a quedar con el índice por default de Pandas. 


* UBICACIÓN 

La columna **PLACE_WITH_PARENT_NAMES** engloba el país, la provincia, el barrio y la zona donde esta. Tiene el 100% de valores y no tiene ningún nulo con lo cual utilizaremos la función SPLIT y REGEX para ir extrayendo paso a paso los elementos de la misma e iremos poniendo a cada uno de ellos en una columna nueva.

Descartaremos las columnas **PLACE_NAME, COUNTRY_NAME y STATE_NAME** ya que por lo que estuvimos analizando podremos sacar esa información e incluso mas de la columna **PLACE_WITH_PARENT_NAMES**


+ GEOLOCALIZACION 

En cuanto a las columnas **GEONAMES_ID, LAT-LON, LAT  y LON** haremos un drop de todas ellas  debido a que son prescindibles en nuestra estrategia para determinar el precio del metro cuadrado de los inmuebles.


+ PRECIO

Al analizar las columnas **PRICE, PRICE_APROX_LOCAL_CURRENCY y PRICE_APROX_USD**, las tres son idénticas, solamente están en una moneda distinta, contienen 100810 valores y 20410 de nulos (porcentaje de nulos de 16,84%). Razón por la cual optaremos por mantener solamente la que se encuentra en dólares para estandarizar con los números que podemos obtener de la columna Description y Title.


+ SUPERFICIE 

La principal idea es rellenar la columna **SURFACE_TOTAL_IN_M2** mediante un REGEX sobre las columnas Description y Title, intentando rellenarla lo mas posible y si esto no alcanza vamos a aplicar los valores de la columna de la superficie cubierta siempre que no exista ningún valor en la columna de superficie total.
Por ello no dropearemos la columna de **SURFACE_COVERED_IN_M2** porque posiblemente la utilicemos para complementar los valores faltantes de la total.


+ PRECIO POR METRO CUADRADO 

La estrategia que vamos a desplegar implica logra conseguir la mayor cantidad de valores de las columnas de superficie total y el precio de los inmuebles en dólares, por lo que dropearemos las columnas **PRICE_USD_PER_M2 y PRICE_PER_M2**


+ CARACTERISTICAS DE LOS INMUEBLES

Primero a la hora de analizar la columna **FLOOR** veremos que la cantidad de plantas que tiene el inmueble no es un dato que nos sirva para el modelo que intentamos lograr con lo cual la dropearemos. Situación parecida sucede con la columna **EXPENSES** con lo cual haremos lo mismo.
Por otro lado, la columna creemos que es muy importante rescatar la cantidad de habitaciones que tienen los inmuebles por lo cual intentaremos rellenar **ROOMS** haciendo un REGEX en las columnas Description y Title.


+ INFO DE LOS INMUEBLES

Conservaremos la columna **PROPERTY_TYPE** ya que creemos que saber el tipo de propiedad puede ser una variable a tener en cuenta a la hora de poder predecir el valor de un inmueble.

Al indagar en las columnas **DESCRIPTION y TITLE** pudimos notar que las mismas contenían mucha y variada información con lo cual les aplicaremos un REGEX a ambas para sacar de ellas datos sobre: cantidad de ambientes, metros cuadrados y precio en usd.

Por último, dropearemos las columnas **IMAGE_THUMBNAI, OPERATION y PROPERATI_URL** ya que ninguna de ellas tiene información relevante para poder llevar a cabo nuestro objetivo.



In [3]:
#data.drop(columns=['Unnamed: 0','operation','place_name','country_name','price','currency','price_aprox_local_currency','expenses','floor','properati_url','image_thumbnail'],inplace=True)

# 2.  Limpieza en particular de la columna place_with_parent_names


### 2.1Teniendo en cuenta que esta variable engloba toda la informacion referida a la ubicacion decidimos filtrarla para extraer las categorias: 
* PAIS
* PROVINCIA 
* BARRIO
* ZONA 



In [4]:
data_places = data.place_with_parent_names
data_places

0                     |Argentina|Capital Federal|Mataderos|
1               |Argentina|Bs.As. G.B.A. Zona Sur|La Plata|
2                     |Argentina|Capital Federal|Mataderos|
3                       |Argentina|Capital Federal|Liniers|
4         |Argentina|Buenos Aires Costa Atlántica|Mar de...
                                ...                        
121215                 |Argentina|Capital Federal|Belgrano|
121216    |Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...
121217            |Argentina|Capital Federal|Villa Urquiza|
121218    |Argentina|Buenos Aires Costa Atlántica|Mar de...
121219                          |Argentina|Capital Federal|
Name: place_with_parent_names, Length: 121220, dtype: object

In [5]:
#Aplicamos str.split(se='|') para quedarnos con los valores entre |
#El argumento expand= True crea un dataFrame con los valores que encuentra x posición.

elementos_data_places = data_places.str.split('|',expand=True)
elementos_data_places.head(3)

,0,1,2,3,4,5,6
0,,Argentina,Capital Federal,Mataderos,,None,None
1,,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,,None,None
2,,Argentina,Capital Federal,Mataderos,,None,None


In [6]:
#Si nos fijamos, una columna entera es una serie entonces podemos agregarla sin problema al DF orginal
type(elementos_data_places[1])

pandas.core.series.Series

### 2.2 Unimos el data frame creado por el split con el dataFrame original creando un nuevo DataSet

In [7]:
data_nuevo = pd.concat([data,elementos_data_places],axis=1,join='inner')

In [8]:
data_nuevo.head(3)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,description,title,image_thumbnail,0,1,2,3,4,5,6
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,,Argentina,Capital Federal,Mataderos,,None,None
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,,None,None
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...,,Argentina,Capital Federal,Mataderos,,None,None


### 2.3 Nulos y porcentajes de nulos 

In [9]:
data_nuevo.isnull().sum()

Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
0                                  0
1

In [10]:
((data_nuevo.isnull().sum()/data.shape[0]) * 100).round(2)

Unnamed: 0                     0.00
operation                      0.00
property_type                  0.00
place_name                     0.02
place_with_parent_names        0.00
country_name                   0.00
state_name                     0.00
geonames_id                   15.44
lat-lon                       42.53
lat                           42.53
lon                           42.53
price                         16.84
currency                      16.84
price_aprox_local_currency    16.84
price_aprox_usd               16.84
surface_total_in_m2           32.44
surface_covered_in_m2         16.42
price_usd_per_m2              43.39
price_per_m2                  27.69
floor                         93.48
rooms                         60.91
expenses                      88.23
properati_url                  0.00
description                    0.00
title                          0.00
image_thumbnail                2.57
0                              0.00
1                           

### 2.4 Analizamos columnas nuevas
Ahora sabemos el porcentaje de nulos de cada columna y sabemos que la columna '0' hace referencia a los indices del DF creado por el split y ademas sabemos que la columna '1' hace referencia al Pais y en todos los casos es Argentina. Tambien las columnas '5' y '6' darian informacion mas especifica acerca de la ubicacion de dicho lugar y al tener un valor de nulos tan alto y al saber que mucha informacion faltante no podra ser completada, decidimos eliminarlas

### 2.5  Renombramos columnas y eliminamos las que no sirven por la cantidad de nulos que tienen.

In [11]:
data_nuevo.rename(columns={0:'indice',1:'Pais',2:'Provincia',3:'Barrio',4:'Zona',5:'elim1',6:'elim2'},inplace=True)

In [12]:
#Eliminamos las columnas que estan de más
data_nuevo.drop(columns=['indice','Pais','elim1','elim2'],inplace=True)

### 2.6 Trabajamos sobre las nuevas columnas para sacar informacion que puede ser util para otras columnas

#### Columna Provincia y Zona
Aplicamos regex sobre columna Prov para sacar valores para obtener los valores de "zona" y completar los valores faltantes en dicha columna

In [13]:
#Completamos valores vacios de Zona con Nan
data_nuevo.Zona.replace('',np.NaN,inplace=True)

In [14]:
# Buscamos valores de Zona 
# Aplico regex para extraer la zona especifica dentro de la columna Zona
# Genero el patron
zona_pattern = "(Zona\s[A-Za-z]+)"

# Compilo el patrón
zona_pattern_regex = re.compile(zona_pattern)

# Genero objeto match
zona_match_result = data_nuevo.Provincia.apply(lambda x: zona_pattern_regex.search(x))

In [15]:
zona_match_result

0                                                      None
1         <re.Match object; span=(14, 22), match='Zona S...
2                                                      None
3                                                      None
4                                                      None
                                ...                        
121215                                                 None
121216    <re.Match object; span=(14, 24), match='Zona N...
121217                                                 None
121218                                                 None
121219                                                 None
Name: Provincia, Length: 121220, dtype: object

In [16]:
# Genero boolean mask sin missings values
notnull_zona_match_mascara = zona_match_result.notnull()

In [17]:
notnull_zona_match_mascara

0         False
1          True
2         False
3         False
4         False
          ...  
121215    False
121216     True
121217    False
121218    False
121219    False
Name: Provincia, Length: 121220, dtype: bool

In [18]:
#Creamos una nueva columna aplicando los match econtrados de la colulma title x group(0)
data_nuevo['Zona_Prov'] = zona_match_result[notnull_zona_match_mascara].apply(lambda x: x if x is np.NaN else x.group(0))

In [19]:
#Cantidad de nulos
data_nuevo.Zona_Prov.isnull().sum()

72386

In [20]:
#Porcentaje de Nulos
(data_nuevo.Zona_Prov.isnull().sum()/data_nuevo.shape[0]) * 100

59.71456855304405

### 2.7 Conclusión

### Luego de realizar el analisis sobre la variable **"place_with_parent_names"** decidimos:

* PAIS = Dropearla porque todos los inmuebles están en Argentina.
* PROVINCIA = Conservarla obtuvimos un 100% de datos
* BARRIO= Conservarla obtuvimos un 100% de datos
* ZONA = Dropearla, ya que a pesar de haber realizado el REGEX sobre la columna title para obtener datos sobre la zona, la misma tiene un 59% de porcentaje de nulos.


### 2.8  Eliminamos columna Zona y Zona_Prov
No pudimos extraer la informacion necesaria para poder completar los valores faltantes y por eso mismo decidimos eliminar dichas columnas

In [21]:
data_nuevo.drop(columns =['Zona','Zona_Prov'],inplace=True)

data_nuevo.head(1)


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,Provincia,Barrio
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,1550.0,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,Capital Federal,Mataderos


# 3. Outliers 

### 3.1 OUTLIERS SUPERFICIE TOTAL Y CUBIERTA

In [22]:
data_nuevo.loc[:,['state_name','property_type','surface_covered_in_m2','surface_total_in_m2']].isnull().sum()

state_name                   0
property_type                0
surface_covered_in_m2    19907
surface_total_in_m2      39328
dtype: int64

In [23]:
data_nuevo.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail', 'Provincia', 'Barrio'],
      dtype='object')

In [24]:
#Copie lo que hicieron los chicos para quedarme con los m2 de description
data_description_serie = data_nuevo.description

# Convertimos a str
data_nuevo["description"] = data_description_serie.astype(str)

# Unificamos tipo de formato (a mayuscula)
data_upper = data_description_serie.apply(lambda x: str(x).upper())
# Comprobamos data_nuevos que se hayan pasado a mayuscula

In [26]:
# Creamos el patron de m2 description
m2_pattern = '(?P<numbers>\d?\d?\d?.?\d?\d?\d)\s?(?P<m2>METROS\s?CUADRADOS|M2|M²|MTS2$)'

# Compilo el patron
m2_pattern_regex = re.compile(m2_pattern)

# Generamos el objeto match
m2_match_result = data_upper.apply(lambda x: m2_pattern_regex.search(x))

# Creamos una boolean mask sin missings values
m2_match_notnull = m2_match_result.notnull()

# Generamos el objeto matc sin missings values
data_nuevo['m2_desc'] = m2_match_result[m2_match_notnull].apply(lambda x: x if x is np.NaN else x.group('numbers'))

In [27]:
data_nuevo.head(2)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,Provincia,Barrio,m2_desc
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...,Capital Federal,Mataderos,NaN
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...,Bs.As. G.B.A. Zona Sur,La Plata,NaN


In [28]:
#Mascara para ver donde m2_mayor a m2_total y que en la nueva columna no haya nulo
mascara = (data_nuevo.surface_covered_in_m2 > data_nuevo.surface_total_in_m2) & (data_nuevo.m2_desc.notnull())
mascara.sum()

237

In [29]:
data_nuevo.loc[mascara,['surface_covered_in_m2','surface_total_in_m2','m2_desc']]

,surface_covered_in_m2,surface_total_in_m2,m2_desc
1397,89.0,80.0,80
1802,240.0,160.0,(32
3884,36.0,34.0,36
3973,259.0,255.0,255.85
4121,710.0,156.0,91
...,...,...,...
116883,235.0,166.0,235
118327,135.0,86.0,000 86
119784,297.0,234.0,297
119830,650.0,336.0,650


### 3.1.1 Outlier: Eliminar deptos mayor a 600mts2 sup total y cubierta

In [30]:
#Generamos mascara xa m2_totales
mascara_depto_outlier = (data_nuevo.property_type == 'apartment') & (data_nuevo.surface_total_in_m2 > 600)
mascara_depto_outlier.sum()

100

In [31]:
#Nos fimjamos la cant de nulos iniciales
data_nuevo.surface_total_in_m2.isnull().sum()

39328

In [32]:
#Reemplazamos los valores de m2_total que cumplan la mascara con Nan 
data_nuevo.loc[mascara_depto_outlier,'surface_total_in_m2'] = np.NaN

In [33]:
#Nos fijamos los nulos y Sumo 100 que es lo esperado
data_nuevo.surface_total_in_m2.isnull().sum()

39428

In [35]:
#Generamos la mascara para m2_covered
mascara_depto_outlier2 = (data_nuevo.property_type == 'apartment') & (data_nuevo.surface_covered_in_m2 > 600)
mascara_depto_outlier2.sum()

80

In [36]:
data_nuevo.loc[:,['state_name','property_type','surface_covered_in_m2','surface_total_in_m2']].isnull().sum()

state_name                   0
property_type                0
surface_covered_in_m2    19907
surface_total_in_m2      39428
dtype: int64

In [37]:
#Nos fimjamos la cant de nulos iniciales
data_nuevo.surface_covered_in_m2.isnull().sum()

19907

In [38]:
#Reemplazamos los valores de m2_covered que cumplan la mascara con Na
data_nuevo.loc[mascara_depto_outlier2,'surface_covered_in_m2'] = np.NaN

In [39]:
data_nuevo.loc[:,['state_name','property_type','surface_covered_in_m2','surface_total_in_m2']].isnull().sum()

state_name                   0
property_type                0
surface_covered_in_m2    19987
surface_total_in_m2      39428
dtype: int64

In [40]:
#Sumo 80 que es lo esperado
data_nuevo.surface_covered_in_m2.isnull().sum()

19987

### 3.1.2. Casos donde m2_total es menor a m2_cubierto

In [41]:
data_nuevo.loc[:,['state_name','property_type','surface_covered_in_m2','surface_total_in_m2']].isnull().sum()

state_name                   0
property_type                0
surface_covered_in_m2    19987
surface_total_in_m2      39428
dtype: int64

In [42]:
#Vemos la cantidad de nulos iniciales
data_nuevo.loc[:,['surface_covered_in_m2','surface_total_in_m2']].isnull().sum()

surface_covered_in_m2    19987
surface_total_in_m2      39428
dtype: int64

In [43]:
#Generamos la mascara
mascara = data_nuevo.surface_covered_in_m2 > data_nuevo.surface_total_in_m2 
mascara.sum()

1094

In [44]:
##Reemplaxzo en ambas columnas con nan con los valores que cumplen con la mascara
data_nuevo.loc[mascara,['surface_covered_in_m2','surface_total_in_m2']] = np.NaN

In [45]:
#Imprimimos nulos y deberia sumar 1094 en ambos casos (ya que cumplen con la primer condicion M2_total < m2_cubierto)
data_nuevo.loc[:,['surface_covered_in_m2','surface_total_in_m2']].isnull().sum()

surface_covered_in_m2    21081
surface_total_in_m2      40522
dtype: int64

### CONCLUSIÓN:

Decidimos reemplazar por NaN cuando la columna de superficie cubierta es mayor a la columna de superficie total(no tiene logica que sea asi). Cuando esto ocurre, reemplazamos el valor de las dos columnas por NaN. 

Analizando esos valores en base a la columna title y description percibimos que no valia la pena salvar los valores por la reducidad cantidad que obtuvimos con Regex.

Para los departamentos tomamos la decision de pegar el valor de la columna superficie total cuando el valor de la columna superficie cubierta esta ausente, y viceversa. Esto lo hacemos solo en el caso de departamentos porque alli normalmente no varian de forma significativa las superficies. En el resto de los tipos de propiedad no podemos usar este supuesto.

### Remplazamos solo en depto los valores que falte el m2_total y no m2_covered y viceversa

In [46]:
#Vemos la cantidad de nulos iniciales
data_nuevo.loc[:,['surface_covered_in_m2','surface_total_in_m2']].isnull().sum()

surface_covered_in_m2    21081
surface_total_in_m2      40522
dtype: int64

In [47]:
#Mascara de m2_total nulo y m2_covered completo solo en depto
mascara_m2_total_nulo = (data_nuevo.surface_total_in_m2.isnull() & data_nuevo.surface_covered_in_m2.notnull()) & (data_nuevo.property_type == 'apartment')
print(mascara_m2_total_nulo.sum())

#Para chequear que haya funcionado bien
data_nuevo.loc[mascara_m2_total_nulo,:].groupby('property_type').size()

13696


property_type
apartment    13696
dtype: int64

In [48]:
#Macscara de m2_total completo y m2_covered nulo solo en depto
mascara_m2_covered_nulo = (data_nuevo.surface_total_in_m2.notnull() & data_nuevo.surface_covered_in_m2.isnull()) & (data_nuevo.property_type == 'apartment')
print(mascara_m2_covered_nulo.sum())

#Para chequear que haya funcionado bien
data_nuevo.loc[mascara_m2_covered_nulo,:].groupby('property_type').size()

5026


property_type
apartment    5026
dtype: int64

In [49]:
#Remplazamos los NaN por el valor de la columna de surface_covered_in_m2 y vemos si la cantidad de nulos disminuyo correctamente
data_nuevo.loc[mascara_m2_total_nulo,'surface_total_in_m2'] = data_nuevo.loc[mascara_m2_total_nulo,'surface_covered_in_m2']
data_nuevo.loc[:,['surface_covered_in_m2','surface_total_in_m2']].isnull().sum()

surface_covered_in_m2    21081
surface_total_in_m2      26826
dtype: int64

In [50]:
#Remplazamos los NaN por el valor de la columna de surface_total_in_m2 y vemos si la cantidad de nulos disminuyo correctamente
data_nuevo.loc[mascara_m2_covered_nulo,'surface_covered_in_m2'] = data_nuevo.loc[mascara_m2_covered_nulo,'surface_total_in_m2']
data_nuevo.loc[:,['surface_covered_in_m2','surface_total_in_m2']].isnull().sum()

surface_covered_in_m2    16055
surface_total_in_m2      26826
dtype: int64

In [51]:
#Volvemos a hacer las mascaras para ver que haya remplazado bien
mascara_m2_covered = (data_nuevo.surface_total_in_m2.notnull() & data_nuevo.surface_covered_in_m2.isnull()) & (data_nuevo.property_type == 'apartment')
mascara_m2_covered.sum()

0

In [52]:
#Volvemos a hacer las mascaras para ver que haya remplazado bien
mascara_m2_total =(data_nuevo.surface_total_in_m2.isnull() & data_nuevo.surface_covered_in_m2.notnull()) & (data_nuevo.property_type == 'apartment')
mascara_m2_total.sum()

0

### Outliers: menos de 10mts los eliminamos tanto en total como en cubierta¶

In [53]:
#Generamos la mascara para m2_total
mascara_menos_10 = (data_nuevo.surface_total_in_m2 < 10)
print(mascara_menos_10.sum())
data_nuevo.loc[mascara_menos_10,:].groupby('property_type').size().sort_values(ascending=False)

518


property_type
apartment    385
house        114
store         11
PH             8
dtype: int64

In [54]:
#Vemos la cantidad de nulos
data_nuevo.isnull().sum().sort_values(ascending=False)

floor                         113321
expenses                      106958
m2_desc                        91179
rooms                          73830
price_usd_per_m2               52603
lat-lon                        51550
lat                            51550
lon                            51550
price_per_m2                   33562
surface_total_in_m2            26826
currency                       20411
price                          20410
price_aprox_local_currency     20410
price_aprox_usd                20410
geonames_id                    18717
surface_covered_in_m2          16055
image_thumbnail                 3112
place_name                        23
Barrio                             0
properati_url                      0
description                        0
title                              0
Provincia                          0
state_name                         0
country_name                       0
place_with_parent_names            0
property_type                      0
o

In [55]:
#Reemplazamos en m2_total con NaN los valores que cumplan con la mascara
data_nuevo.loc[mascara_menos_10,'surface_total_in_m2'] = np.NaN

In [56]:
#Vemos los nulos y tiene que aumentar solo en m2_total
data_nuevo.isnull().sum().sort_values(ascending=False)

floor                         113321
expenses                      106958
m2_desc                        91179
rooms                          73830
price_usd_per_m2               52603
lat-lon                        51550
lat                            51550
lon                            51550
price_per_m2                   33562
surface_total_in_m2            27344
currency                       20411
price                          20410
price_aprox_local_currency     20410
price_aprox_usd                20410
geonames_id                    18717
surface_covered_in_m2          16055
image_thumbnail                 3112
place_name                        23
Barrio                             0
properati_url                      0
description                        0
title                              0
Provincia                          0
state_name                         0
country_name                       0
place_with_parent_names            0
property_type                      0
o

In [57]:
#Para chequear de nuevo probamos la mascara y nos tiene que dar 0
mascara_menos_10 = (data_nuevo.surface_total_in_m2 < 10)
print(mascara_menos_10.sum())

0


In [59]:
#Generamos la mascara para m2_covered
mascara_menos_10_covered = (data_nuevo.surface_covered_in_m2 < 10)
print(mascara_menos_10_covered.sum())
data_nuevo.loc[mascara_menos_10_covered,:].groupby('property_type').size().sort_values(ascending=False)

684


property_type
apartment    454
house        163
store         35
PH            32
dtype: int64

In [60]:
#Reemplazamos en m2_covered con NaN los valores que cumplan con la mascara
data_nuevo.loc[mascara_menos_10_covered,'surface_covered_in_m2'] = np.NaN

In [61]:
#Vemos los nulos y tiene que aumentar solo en m2_covered
data_nuevo.isnull().sum().sort_values(ascending=False)

floor                         113321
expenses                      106958
m2_desc                        91179
rooms                          73830
price_usd_per_m2               52603
lat-lon                        51550
lat                            51550
lon                            51550
price_per_m2                   33562
surface_total_in_m2            27344
currency                       20411
price                          20410
price_aprox_local_currency     20410
price_aprox_usd                20410
geonames_id                    18717
surface_covered_in_m2          16739
image_thumbnail                 3112
place_name                        23
Barrio                             0
properati_url                      0
description                        0
title                              0
Provincia                          0
state_name                         0
country_name                       0
place_with_parent_names            0
property_type                      0
o

In [62]:
#Para chequear de nuevo probamos la mascara y nos tiene que dar 0
mascara_menos_10_covered = (data_nuevo.surface_covered_in_m2 < 10)
print(mascara_menos_10_covered.sum())

0


### 3.4 Outliers Precios en USD 

### 3.4.1. Analisis de valores maximos y minimos de la columna price_aprox_usd (columna de referencia de precio)

In [63]:
# Visualizo los missings values antes de la operacion
data_nuevo.price_aprox_usd.isnull().sum()

20410

In [64]:
# Creo las boolean masks para filtrar por departamento
data_apartment_mask =  data_nuevo.property_type =='apartment'

# Filtro por tipo de propiedad
data_apartment = data_nuevo[data_apartment_mask]

In [65]:
# Creo la serie de precios en dolares para departamentos (solo a efectos de visualizacion)
price_aprox_usd_apartments = data_apartment.price_aprox_usd

In [66]:
# Visualizo los maximos y minimos con describe antes de la operacion
describe_with_outliers = round(price_aprox_usd_apartments.describe(),2)
describe_with_outliers

count       59616.00
mean       187406.88
std        316218.35
min          4666.62
25%         80000.00
50%        119000.00
75%        190000.00
max      46545445.00
Name: price_aprox_usd, dtype: float64

In [67]:
# Visualizo los maximos y minimos con mas detalle usando sort_values
max_apartment = price_aprox_usd_apartments.sort_values(ascending=False)[:68]
print(max_apartment,'\n')
min_apartment = price_aprox_usd_apartments.sort_values(ascending=True)[:68]
print(min_apartment)

24548    46545445.0
44685     8500000.0
44684     8000000.0
73029     8000000.0
26637     6809000.0
            ...    
25163     2800000.0
81753     2800000.0
58143     2800000.0
26568     2788520.0
18186     2700000.0
Name: price_aprox_usd, Length: 68, dtype: float64 

34666      4666.62
52421      5000.00
4399       5000.00
57415      5047.53
102031     5047.53
            ...   
26933      9500.00
115717     9534.22
115223     9814.64
59881      9916.58
10190     10000.00
Name: price_aprox_usd, Length: 68, dtype: float64


In [68]:
# Visualizo los valores maximos con la poscion de los valores obtenidos en el punto anterior
data_nuevo.loc[[24548,44685,44685,73029,26637,58175,52789,53048,53116,52788],['price_aprox_usd','property_type','Provincia','Barrio','description','title']]

,price_aprox_usd,property_type,Provincia,Barrio,description,title
24548,46545445.0,apartment,Capital Federal,San Telmo,dsdffhghvcvbcdfdfghdfhghkjhkhjklhjkfgvghfghnfg...,Departamento venta
44685,8500000.0,apartment,Capital Federal,Palermo,CONTACTO: MARTIN PINUS 15- EXCELENTISIMA PROPI...,LE PARC ALCORTA - PISO MUY ALTO DE 410 M2 CON ...
44685,8500000.0,apartment,Capital Federal,Palermo,CONTACTO: MARTIN PINUS 15- EXCELENTISIMA PROPI...,LE PARC ALCORTA - PISO MUY ALTO DE 410 M2 CON ...
73029,8000000.0,apartment,Capital Federal,Palermo,VENTA DEPARTAMENTO PISO DE 2 UNIDADES ANEXADAS...,Departamento en Palermo
26637,6809000.0,apartment,Capital Federal,Boedo,Departamento 1 ambiente divisible Depto en CU...,"Tu depa, ingresa con $ 29.000 y cuotas en pesos"
58175,6000000.0,apartment,Capital Federal,Nuñez,TRIPLEX EN FORUM ALCORTA! VISTA ESPECTACULAR A...,FORUM ALCORTA TRIPLEX
52789,5500000.0,apartment,Capital Federal,Nuñez,Espectacular Penthouse en Forum Alcorta! Trilp...,Espectacular Penthouse en Forum Alcorta!
53048,5500000.0,apartment,Capital Federal,Belgrano,Espectacular Penthouse en Forum Alcorta! Trilp...,Espectacular Penthouse en Forum Alcorta!
53116,5500000.0,apartment,Capital Federal,Nuñez,Espectacular Penthouse en Forum Alcorta! Trilp...,Espectacular Penthouse en Forum Alcorta!
52788,5500000.0,apartment,Capital Federal,Belgrano,Espectacular Penthouse en Forum Alcorta! Trilp...,Espectacular Penthouse en Forum Alcorta!


In [69]:
# Visualizo los valores minimos con la poscion de los valores obtenidos en el sort_values
data_nuevo.loc[[34666,52421,4399,57415,57415,27265,108145,20133,113692,53678,10190],['price_aprox_usd','property_type','Provincia','Barrio','description','title']]

,price_aprox_usd,property_type,Provincia,Barrio,description,title
34666,4666.62,apartment,Santa Fe,Rosario,PLAN BAUEN PILAY 1 DORMITORIO SIN ADJUDICAR CO...,Departamento venta
52421,5000.00,apartment,Bs.As. G.B.A. Zona Oeste,Tres de Febrero,Corredor Responsable: Mauro Marvisi - CMCPSI 5...,"MINIMO ADELANTO, SIN INTERÉS Y CUOTAS FIJAS."
4399,5000.00,apartment,Bs.As. G.B.A. Zona Oeste,Tres de Febrero,Corredor Responsable: Mauro Marvisi - CMCPSI 5...,"SIN ADELANTO, SIN INTERÉS Y CUOTAS FIJAS."
57415,5047.53,apartment,Bs.As. G.B.A. Zona Norte,San Fernando,San Fernando: departamento de dos ambientes e...,DEPARTAMENTO EN VENTA
57415,5047.53,apartment,Bs.As. G.B.A. Zona Norte,San Fernando,San Fernando: departamento de dos ambientes e...,DEPARTAMENTO EN VENTA
27265,5047.53,apartment,Santa Fe,Rosario,Corredor Responsable: Jesica Tobio - CI Mat. N...,Venta Plan Coop. De Viviendas Rosario
108145,5103.61,apartment,Capital Federal,Palermo,CODIGO: 916-69637 ubicado en: SCALABRINI ORTIZ...,"Departamento en Venta en Palermo Soho, Capital..."
20133,5103.61,apartment,Capital Federal,Villa Crespo,CODIGO: 6375 ubicado en: Vera 1100 - Publicad...,Venta 1 Ambiente al Frente Vera 1100
113692,5159.69,apartment,Bs.As. G.B.A. Zona Oeste,La Matanza,CODIGO: ubicado en: Espora 900 - Publicado p...,Espora 900
53678,5191.62,apartment,Capital Federal,Parque Chacabuco,DUEÑO VENDE 2amb C/amplio Patio\t\t* Amplio de...,Departamento venta


### 3.4.2 Calculamos y filtramos los outliers de los valores minimos y maximos

**Alternativa 1**: Filtrar outliers de acuerdo a los valores de corte definidos

En funcion de la informacion obtenida en el punto anterior contemplamos el criterio de definicion de valores de corte para establecer los outliers y posteriormente eliminar los mismos del dataset. Los criterios de definicion que establcemos son:

* Valores minimos: Propiedades con valores inferiores a USD 10.000
* Valores maximos: Propiedad con valor de USD 46.545.445

In [70]:
# Filtro el outlier maximo reemplazando al mismo por NaN
data_nuevo.loc[24548,'price_aprox_usd'] = np.NaN

In [71]:
# Calculo la boolean mask para los valores de corte minimos
price_usd_min_mask = (data_nuevo.price_aprox_usd < 10000)

# Guardo los outliers minimos en variables solo a efectos de visualizar
price_usd_min = data_nuevo.loc[price_usd_min_mask]

print(price_usd_min_mask.sum(),'\n')

107 



In [72]:
# Visualizo los missings values antes de la operacion
data_nuevo.price_aprox_usd.isnull().sum()

20411

In [73]:
price_usd_min.sort_values(by = 'price_aprox_usd', ascending = False)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,Provincia,Barrio,m2_desc
59881,59881,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.9528056,-60.6397661",-32.952806,...,NaN,7.0,NaN,http://www.properati.com.ar/19y63_venta_depart...,Se vende plan de departamento de 3 dormitorios...,Departamento venta,https://thumbs4.properati.com/4/-i9GWZ9EtUPmbF...,Santa Fe,Rosario,NaN
115223,115223,sell,apartment,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,"-31.426905,-64.230881",-31.426905,...,NaN,NaN,NaN,http://www.properati.com.ar/1cawf_venta_depart...,Liquido plan de depto de 1 dormitorio con coch...,Euro Mayor Antigua Cerveceria.,https://thumbs4.properati.com/8/G9_NwRdLMcx9NI...,Córdoba,Córdoba,NaN
115717,115717,sell,apartment,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,NaN,NaN,...,NaN,NaN,NaN,http://www.properati.com.ar/1cbi8_venta_depart...,3d + dep. 3b liv/com coc/com Calef. x rad. Bal...,VENDO - Nva. Cba 3d. s/Bs. As. 400,https://thumbs4.properati.com/7/jn-ClncZLIXkzu...,Córdoba,Córdoba,NaN
9665,9665,sell,house,Ingeniero Pablo Nogués,|Argentina|Bs.As. G.B.A. Zona Norte|Malvinas A...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,...,NaN,NaN,NaN,http://www.properati.com.ar/16599_venta_casa_i...,CODIGO: 7 ubicado en: Juana Manuela Gorritti -...,Casa en Venta con Anticipo y Cuotas en Tortugu...,https://thumbs4.properati.com/6/baljQaRJQoHBNV...,Bs.As. G.B.A. Zona Norte,Malvinas Argentinas,200
26932,26932,sell,apartment,Flores,|Argentina|Capital Federal|Flores|,Argentina,Capital Federal,3433918.0,"-34.6324508,-58.4706827",-34.632451,...,NaN,2.0,NaN,http://www.properati.com.ar/17rrw_venta_depart...,Excelente Metros de Subte L?nea *A* Frente Su...,metros de subte linea A financiacion BANCARIA,https://thumbs4.properati.com/2/_je5QjEsSV4aFn...,Capital Federal,Flores,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52421,52421,sell,apartment,Caseros,|Argentina|Bs.As. G.B.A. Zona Oeste|Tres de Fe...,Argentina,Bs.As. G.B.A. Zona Oeste,3862356.0,"-34.602217,-58.564221",-34.602217,...,NaN,3.0,NaN,http://www.properati.com.ar/19j3x_venta_depart...,Corredor Responsable: Mauro Marvisi - CMCPSI 5...,"MINIMO ADELANTO, SIN INTERÉS Y CUOTAS FIJAS.",https://thumbs4.properati.com/0/2V1QeEncfKcbhX...,Bs.As. G.B.A. Zona Oeste,Tres de Febrero,NaN
4399,4399,sell,apartment,Caseros,|Argentina|Bs.As. G.B.A. Zona Oeste|Tres de Fe...,Argentina,Bs.As. G.B.A. Zona Oeste,3862356.0,"-34.609431,-58.566785",-34.609431,...,NaN,2.0,NaN,http://www.properati.com.ar/15mqu_venta_depart...,Corredor Responsable: Mauro Marvisi - CMCPSI 5...,"SIN ADELANTO, SIN INTERÉS Y CUOTAS FIJAS.",https://thumbs4.properati.com/4/6GKCqfr3Xa7F2P...,Bs.As. G.B.A. Zona Oeste,Tres de Febrero,NaN
59875,59875,sell,store,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-37.9997575,-57.5509063",-37.999758,...,NaN,1.0,NaN,http://www.properati.com.ar/19y3o_venta_local_...,Exelente oportunidad. Venta de fondo de comerc...,Local venta,https://thumbs4.properati.com/3/3OYvzzHcY5S0hJ...,Buenos Aires Costa Atlántica,Mar del Plata,*60
34666,34666,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-38.416097,-63.616672",-38.416097,...,NaN,2.0,NaN,http://www.properati.com.ar/18cia_venta_depart...,PLAN BAUEN PILAY 1 DORMITORIO SIN ADJUDICAR CO...,Departamento venta,https://thumbs4.properati.com/5/t7GjDmf0BY5oX_...,Santa Fe,Rosario,NaN


In [74]:
# Calculo la boolean mask para excluir outliers en surface_total_in_m2
not_outliers_price_min = np.logical_not(price_usd_min_mask)

# Filtro surface_total_in_m2 excluyendo los outliers
data_nuevo['price_aprox_usd'] = data_nuevo.price_aprox_usd[not_outliers_price_min]

In [75]:
# Chequeo que los outliers hayan sido correctamente filtrados
data_nuevo.loc[24548,'price_aprox_usd']

nan

In [76]:
data_nuevo.loc[price_usd_min_mask]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,floor,rooms,expenses,properati_url,description,title,image_thumbnail,Provincia,Barrio,m2_desc
836,836,sell,store,Tolosa,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|Tol...,Argentina,Bs.As. G.B.A. Zona Sur,3427714.0,"-34.8866047479,-57.9694639519",-34.886605,...,NaN,1.0,NaN,http://www.properati.com.ar/15g6s_venta_local_...,"Venta de Negocio en Tolosa, La Plata116 entre...",NEGOCIO EN VENTA,https://thumbs4.properati.com/9/DHgL8aLPFIi1RC...,Bs.As. G.B.A. Zona Sur,La Plata,NaN
4399,4399,sell,apartment,Caseros,|Argentina|Bs.As. G.B.A. Zona Oeste|Tres de Fe...,Argentina,Bs.As. G.B.A. Zona Oeste,3862356.0,"-34.609431,-58.566785",-34.609431,...,NaN,2.0,NaN,http://www.properati.com.ar/15mqu_venta_depart...,Corredor Responsable: Mauro Marvisi - CMCPSI 5...,"SIN ADELANTO, SIN INTERÉS Y CUOTAS FIJAS.",https://thumbs4.properati.com/4/6GKCqfr3Xa7F2P...,Bs.As. G.B.A. Zona Oeste,Tres de Febrero,NaN
6720,6720,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.9100582,-60.6874369",-32.910058,...,NaN,2.0,NaN,http://www.properati.com.ar/15vcm_venta_depart...,BV Rondeau / Nancen: A metros del Shoping Port...,Venta departamento de 1 dormitorios a estrenar...,https://thumbs4.properati.com/9/LbRdgRWsLUiDR5...,Santa Fe,Rosario,"37,29"
7440,7440,sell,house,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,"-38.0947287121,-57.5552277112",-38.094729,...,NaN,4.0,NaN,http://www.properati.com.ar/15xy8_venta_casa_m...,"EXCELENTE CABAÑA 3 DORM, 2 BAÑOS, PARQUE, A 30...",CABAÑA 4 AMB. ALFAR,https://thumbs4.properati.com/6/fzHuWELA_ZurQe...,Buenos Aires Costa Atlántica,Mar del Plata,NaN
9665,9665,sell,house,Ingeniero Pablo Nogués,|Argentina|Bs.As. G.B.A. Zona Norte|Malvinas A...,Argentina,Bs.As. G.B.A. Zona Norte,NaN,NaN,NaN,...,NaN,NaN,NaN,http://www.properati.com.ar/16599_venta_casa_i...,CODIGO: 7 ubicado en: Juana Manuela Gorritti -...,Casa en Venta con Anticipo y Cuotas en Tortugu...,https://thumbs4.properati.com/6/baljQaRJQoHBNV...,Bs.As. G.B.A. Zona Norte,Malvinas Argentinas,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115717,115717,sell,apartment,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,NaN,NaN,...,NaN,NaN,NaN,http://www.properati.com.ar/1cbi8_venta_depart...,3d + dep. 3b liv/com coc/com Calef. x rad. Bal...,VENDO - Nva. Cba 3d. s/Bs. As. 400,https://thumbs4.properati.com/7/jn-ClncZLIXkzu...,Córdoba,Córdoba,NaN
116091,116091,sell,house,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,"-31.34361,-64.274445",-31.343610,...,NaN,8.0,NaN,http://www.properati.com.ar/1cbw8_venta_casa_c...,OPORTUNIDAD!! APTO A CREDITO. HERMOSO DUPLEX 2...,"B. LASALLE OPORTUNIDAD!! 3D, APTO CREDITO!!...",https://thumbs4.properati.com/3/oP6NQpxF9UjR0f...,Córdoba,Córdoba,100
116133,116133,sell,house,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,NaN,NaN,...,NaN,11.0,NaN,http://www.properati.com.ar/1cbxw_venta_casa_c...,"VENDO DUPLEX-CASI NUEVO-Bº Alto Verde, próximo...",VENDO-DUPLEX-Bº ALTO VERDE-3DORM 2BÑ LIV COC/C...,https://thumbs4.properati.com/1/bprbHhwaWJ7bfQ...,Córdoba,Córdoba,200
116277,116277,sell,PH,Mar del Plata,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3430863.0,NaN,NaN,...,NaN,3.0,NaN,http://www.properati.com.ar/1cc41_venta_ph_mar...,"P3 ambientes, posee: Living comedor, cocina co...",FALKNER 3700,https://thumbs4.properati.com/9/FQsiR42FWb-iyK...,Buenos Aires Costa Atlántica,Mar del Plata,NaN


In [77]:
# Visualizo los missings values despues de la operacion
data_nuevo.price_aprox_usd.isnull().sum()

20518

**Alternativa 2**: Filtrar los outliers aplicando la regla del rango intercuartil


En este punto aplicamos el criterio estadistico de aplicacion del rango intercuartil y el establecimiento de umbrales para filtrar los outliers que superen los umbrales establecidos

In [ ]:
# Calculo los cuartilos
# q1 = data_nuevo.price_aprox_usd.quantile(0.25)
# q2 = data_nuevo.price_aprox_usd.quantile(0.5)
# q3 = data_nuevo.price_aprox_usd.quantile(0.75)
# q4 = data_nuevo.price_aprox_usd.quantile(1)

# Aplico la regla de rango intercuartil
# iqr = (q3 - q1) * 1.5

# Calculo los umbrales
# upper_threshold = q3 + iqr
# lower_threshold = q1 - iqr

# Calculo los outliers
# outliers_mask_up_ap = (data_nuevo.price_aprox_usd > upper_threshold) & (data_nuevo.property_type=='apartment')
# outliers_mask_down_ap = (data_nuevo.price_aprox_usd < lower_threshold) & (data_nuevo.property_type=='apartment')
#outliers_mask_ap = np.logical_or(outliers_mask_up_ap,outliers_mask_down_ap) # Aplico np.logical_or en lugar de utilizar pipes
#outliers_ap = data_nuevo.price_aprox_usd[outliers_mask_ap]

# Calculo la boolean mask para excluir outliers
#not_outliers_ap = np.logical_not(outliers_mask_ap)

# Filtro movie_votes_count excluyendo los outliers
#price_aprox_usd_filter =  data_nuevo.price_aprox_usd[not_outliers_ap]

In [78]:
# Comparo price_aprox_usd con y sin outliers
#describe_without_outliers = round(price_aprox_usd_filter.describe(),2)
#print(describe_with_outliers,'\n')
#print(describe_without_outliers)

Conclusion : Concluimos que esta alternativa, no resulta conveniente para aplicar en este contexto inicial, ya que podemos incurrir en el riesgo de eliminar datos que pueden resultarnos valiosos para esta o posteriores etapas en nuestro dataset, y, ademas, el dataset bajo analisis no sigue una distribucion normal y expone niveles altos de dispersion de datos. De todas formas, consideramos que este criterio podria ser util para aplicar en posteriores etapas, fundamentalmente despues del proceso de limpieza y/o purificacion de datos.

In [79]:
# Visualizo los missings values despues de la operacion
data_nuevo.price_aprox_usd.isnull().sum()

20518

#  4.Regex a columna "description"

### Vamos a aplicar Regex sobre la columna description para intentar extraer los valores de las variables:

* Rooms 
* Dólares 
* M2
* Tipo propiedad 


### 4.1 Hacemos Regex sobre la columna description para sacar la cantidad de Rooms 

In [80]:
# Pasos para obtener patrones para description
# Creo la serie en description

data_description_serie = data_nuevo.description

# Convertimos a str
data_nuevo["description"] = data_description_serie.astype(str)

# Unificamos tipo de formato (a mayuscula)
data_upper = data_description_serie.apply(lambda x: str(x).upper())
# Comprobamos data_nuevos que se hayan pasado a mayuscula
print(data_upper.head(10))

# Creamos el patron para rooms
room_pattern = "(?P<cantidad>\d\d?)\s(?P<room_ref>\AMB|AMBIENTES|HABITACIONES|HAB|AMB.|HAB.$)"

# Compilo el patron
room_pattern_regex = re.compile(room_pattern)

# Generamos el objeto match
room_match_result = data_upper.apply(lambda x: room_pattern_regex.search(x))

# Creamos una boolean mask sin missings values
room_match_notnull = room_match_result.notnull()

# Generamos el objeto matc sin missings values
data_nuevo['rooms_desc'] = room_match_result[room_match_notnull].apply(lambda x: x if x is np.NaN else x.group('cantidad'))

0    2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...
1    VENTA DE DEPARTAMENTO EN DÉCIMO PISO AL FRENTE...
2    2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...
3    PH 3 AMBIENTES CON PATIO. HAY 3 DEPTOS EN LOTE...
4    DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...
5    CASA EN EL PERÍMETRO DEL BARRIO 338, UBICADA E...
6    MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...
7    EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...
8    EXCELENTE DOS AMBIENTES  ESTRENAR AMPLIO SUPER...
9    MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...
Name: description, dtype: object


In [81]:
# chequeamos los valores unicos que rescatamos para controlar 
data_nuevo.rooms_desc.unique()

array(['2', nan, '3', '1', '4', '5', '7', '00', '6', '30', '15', '50',
       '9', '55', '8', '42', '16', '90', '23', '10', '13', '17', '03',
       '97', '26', '72', '14', '54', '02', '18', '21', '11', '57', '24',
       '12', '44', '71', '53', '25', '20', '81', '62', '73', '22', '33',
       '52', '04', '27', '83', '70', '28', '51', '46', '60', '49', '40',
       '47', '74', '93', '06', '66', '0', '92', '80', '01', '29', '32',
       '56', '65', '84', '36', '64'], dtype=object)

In [82]:
#verificamos la cantidad de valores obtenidos del regex
data_nuevo.rooms_desc.notnull().sum()

39761

In [83]:
# Evaluamos coherencia y correcta aplicacion del match
data_nuevo.loc[:,['description','rooms_desc']]

,description,rooms_desc
0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2
1,Venta de departamento en décimo piso al frente...,NaN
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2
3,PH 3 ambientes con patio. Hay 3 deptos en lote...,3
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,NaN
...,...,...
121215,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,NaN
121216,Excelente e impecable casa en Venta en Las Lom...,NaN
121217,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,NaN
121218,"2 Amb al contrafrente, luminoso. El departame...",2


### 4.2 Hacemos Regex sobre la columna description para sacar los dolares 

In [84]:
# Pasos regex precios description
# Creamos el patron dolares
dollar_pattern = "(?P<dolar>(US\W+)|(U\WD)|(USD))\s?(?P<monto>(\d?\d?\d?.\d?\d\d.\d\d\d))"

# Compilo el patron
dollar_pattern_regex = re.compile(dollar_pattern)

# Genero el objeto match
dollar_match_result = data_upper.apply(lambda x: dollar_pattern_regex.search(x))
print(room_match_result)

# Creo la boolean mask sin missings values
dollar_match_notnull = dollar_match_result.notnull()

# Creo el objeto match sin missings values
data_nuevo['dollar_desc'] = dollar_match_result[dollar_match_notnull].apply(lambda x: x if x is np.NaN else x.group(0))

0         <re.Match object; span=(0, 11), match='2 AMBIE...
1                                                      None
2         <re.Match object; span=(0, 11), match='2 AMBIE...
3         <re.Match object; span=(3, 14), match='3 AMBIE...
4                                                      None
                                ...                        
121215                                                 None
121216                                                 None
121217                                                 None
121218       <re.Match object; span=(0, 6), match='2 AMB '>
121219                                                 None
Name: description, Length: 121220, dtype: object


In [85]:
# Vemos cuantos valores hay en price_aprox_usd antes de la limpieza y posterior relleno de datos
data_nuevo.price_aprox_usd.notnull().sum()

100702

In [86]:
data_nuevo.dollar_desc.notnull().sum()

3924

In [87]:
# Evaluamos coherencia y correcta aplicacion del match
data_nuevo.loc[0:10,['description','dollar_desc']]

,description,dollar_desc
0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",NaN
1,Venta de departamento en décimo piso al frente...,U$D 20.000
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,NaN
3,PH 3 ambientes con patio. Hay 3 deptos en lote...,NaN
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,NaN
5,"Casa en el perímetro del barrio 338, ubicada e...",NaN
6,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,NaN
7,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,NaN
8,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,NaN
9,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,NaN


### 4.3 Hacemos Regex sobre la columna description para sacar los M2 

In [88]:
# Creamos el patron de m2 description
m2_pattern = '(?P<numbers>\d?\d?\d?.?\d?\d?\d)\s?(?P<m2>METROS\s?CUADRADOS|M2|M²|MTS2$)'

# Compilo el patron
m2_pattern_regex = re.compile(m2_pattern)

# Generamos el objeto match
m2_match_result = data_upper.apply(lambda x: m2_pattern_regex.search(x))

# Creamos una boolean mask sin missings values
m2_match_notnull = m2_match_result.notnull()

# Generamos el objeto matc sin missings values
data_nuevo['m2_desc'] = m2_match_result[m2_match_notnull].apply(lambda x: x if x is np.NaN else x.group('numbers'))

In [89]:
data_nuevo['m2_desc'].notnull().sum()

30041

In [90]:
# Evaluamos coherencia y correcta aplicacion del match
data_nuevo.loc[:,['description','m2_desc']]

,description,m2_desc
0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",NaN
1,Venta de departamento en décimo piso al frente...,NaN
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,NaN
3,PH 3 ambientes con patio. Hay 3 deptos en lote...,NaN
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,NaN
...,...,...
121215,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,NaN
121216,Excelente e impecable casa en Venta en Las Lom...,572
121217,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,"43,00"
121218,"2 Amb al contrafrente, luminoso. El departame...",NaN


In [91]:
data_nuevo.surface_total_in_m2.notnull().sum()

93876

In [92]:
data_nuevo.surface_total_in_m2.unique 

<bound method Series.unique of 0          55.0
1           NaN
2          55.0
3           NaN
4          35.0
          ...  
121215    113.0
121216    360.0
121217     46.0
121218     48.0
121219     77.0
Name: surface_total_in_m2, Length: 121220, dtype: float64>

### 4.4 Hacemos Regex sobre la columna description para sacar el tipo de propiedad  

In [93]:
# Creamos el patron de tipo de propiedad para descrption
tproperty_pattern = '(?P<property_type>CASA|DUPLEX|RESIDENCIA|CHALET)'

# Compilo el patron
tproperty_pattern_regex = re.compile(tproperty_pattern)

# Generamos el objeto match
tproperty_match_result = data_upper.apply(lambda x: tproperty_pattern_regex.search(x))

# Creamos una boolean mask sin missings values
tproperty_match_notnull = tproperty_match_result.notnull()

# Generamos el objeto matc sin missings values
data_nuevo['property_desc'] = tproperty_match_result[tproperty_match_notnull].apply(lambda x: x if x is np.NaN else x.group(0))

In [94]:
data_nuevo.loc[:,['description','property_desc']]

,description,property_desc
0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",CASA
1,Venta de departamento en décimo piso al frente...,NaN
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,NaN
3,PH 3 ambientes con patio. Hay 3 deptos en lote...,NaN
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,NaN
...,...,...
121215,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,NaN
121216,Excelente e impecable casa en Venta en Las Lom...,CASA
121217,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,NaN
121218,"2 Amb al contrafrente, luminoso. El departame...",NaN


In [95]:
data_nuevo.property_desc.notnull().sum()

42273

# 5. Regex a la columna "title"

### Vamos a aplicar Regex sobre la columna description para intentar extraer los valores de las variables:

* Rooms 
* Dólares 
* M2
* Tipo propiedad 


### 5.1 Hacemos Regex sobre la columna title para sacar la cantidad de rooms

In [96]:
# Pasos para obtener patrones para title
# Creo la serie en title
data_title_serie = data_nuevo.title

# Convertimos a str
data_nuevo["title"] = data_title_serie.astype(str)

# Unificamos tipo de formato (a mayuscula)
data_upper_title = data_title_serie.apply(lambda x: str(x).upper())
# Comprobamos data_nuevos que se hayan pasado a mayuscula
print(data_upper_title.head(10))

# Creamos el patron para rooms
rooms_pattern = "(?P<cantidad>\d\d?)\s(?P<room_ref>\AMB|AMBIENTES|HABITACIONES|HAB|AMB.|HAB.$)"

# Compilo el patron
rooms_pattern_regex = re.compile(rooms_pattern)

# Generamos el objeto match
rooms_match_result = data_upper_title.apply(lambda x: rooms_pattern_regex.search(x))

# Creamos una boolean mask sin missings values
rooms_match_notnull = rooms_match_result.notnull()

# Generamos el objeto matc sin missings values
data_nuevo['rooms_title'] = rooms_match_result[rooms_match_notnull].apply(lambda x: x if x is np.NaN else x.group('cantidad'))

0                   2 AMB TIPO CASA SIN EXPENSAS EN PB
1    VENTA DEPTO 2 DORM. A ESTRENAR 7 E/ 36 Y 37   ...
2             2 AMB 3ER PISO CON ASCENSOR APTO CREDITO
3                         PH 3 AMB. CFTE. RECICLADO   
4    DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...
5    CASA BARRIO 338. SOBRE CALLE 3 DE CABALLERÍA, ...
6    MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...
7    JOSE HERNANDEZ 1400 MONOAMBIENTE  ESTRENAR CAT...
8    JOSE HERNANDEZ   1400 DOS AMBIENTES ESTRENAR ,...
9           WHITE 7637 - 2 DORMITORIOS CON PATIO      
Name: title, dtype: object


In [97]:
data_nuevo.loc[:,['title','rooms_title']]

,title,rooms_title
0,2 AMB TIPO CASA SIN EXPENSAS EN PB,2
1,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN
2,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,2
3,PH 3 amb. cfte. reciclado,3
4,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,2
...,...,...
121215,Torre Forum Alcorta- Impecable 3 ambientes,3
121216,Ruca Inmuebles | Venta | Lomas de San Isidro |...,NaN
121217,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,NaN
121218,2 amb. C/ dep. de servicio al contrafrente| Re...,2


In [98]:
data_nuevo.rooms_title.notnull().sum()

20974

### 5.2  Hacemos Regex sobre la columna title para los valores de precio en dolares 

In [99]:
# Pasos regex precios en title
# Creamos el patron para dolares
price_pattern = "(?P<dolar>(US\W+)|(U\WD)|(USD))\s?(?P<monto>(\d?\d?\d?.\d?\d\d.\d\d\d))"

# Compilo el patron
price_pattern_regex = re.compile(price_pattern)

# Genero el objeto match
price_match_result = data_upper_title.apply(lambda x: price_pattern_regex.search(x))

# Creo la boolean mask sin missings values
price_match_notnull = price_match_result.notnull()

# Creo el objeto match sin missings values
data_nuevo['price_title'] = price_match_result[price_match_notnull].apply(lambda x: x if x is np.NaN else x.group(0))

In [100]:
data_nuevo.loc[:,['title','price_title']]

,title,price_title
0,2 AMB TIPO CASA SIN EXPENSAS EN PB,NaN
1,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN
2,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,NaN
3,PH 3 amb. cfte. reciclado,NaN
4,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,NaN
...,...,...
121215,Torre Forum Alcorta- Impecable 3 ambientes,NaN
121216,Ruca Inmuebles | Venta | Lomas de San Isidro |...,NaN
121217,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,NaN
121218,2 amb. C/ dep. de servicio al contrafrente| Re...,NaN


In [101]:
data_nuevo.price_title.notnull().sum()

2534

### 5.3 Hacemos Regex sobre la columna title para sacar la cantidad de metros cuadrados.

In [102]:
# Creamos el patron de m2 para title
mts2_pattern = '(?P<numbers>\d?\d?\d?.?\d?\d?\d)\s?(?P<m2>METROS\s?CUADRADOS|M2|M²|MTS2$)'

# Compilo el patron
mts2_pattern_regex = re.compile(mts2_pattern)

# Generamos el objeto match
mts2_match_result = data_upper_title.apply(lambda x: mts2_pattern_regex.search(x))

# Creamos una boolean mask sin missings values
mts2_match_notnull = mts2_match_result.notnull()

# Generamos el objeto matc sin missings values
data_nuevo['m2_title'] = mts2_match_result[mts2_match_notnull].apply(lambda x: x if x is np.NaN else x.group(0))

In [103]:
data_nuevo.loc[:,['title','m2_title']]

,title,m2_title
0,2 AMB TIPO CASA SIN EXPENSAS EN PB,NaN
1,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN
2,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,NaN
3,PH 3 amb. cfte. reciclado,NaN
4,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,NaN
...,...,...
121215,Torre Forum Alcorta- Impecable 3 ambientes,NaN
121216,Ruca Inmuebles | Venta | Lomas de San Isidro |...,NaN
121217,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,NaN
121218,2 amb. C/ dep. de servicio al contrafrente| Re...,NaN


In [104]:
data_nuevo.m2_title.notnull().sum()

6222

### 5.4 Hacemos Regex sobre la columna title para sacar el tipo de propiedad 

In [106]:
# Creamos el patron de tipo de propiedad para title
property_pattern = '(?P<property_type>CASA|DUPLEX|RESIDENCIA|CHALET)'

# Compilo el patron
property_pattern_regex = re.compile(property_pattern)

# Generamos el objeto match
property_match_result = data_upper_title.apply(lambda x: property_pattern_regex.search(x))

# Creamos una boolean mask sin missings values
property_match_notnull = property_match_result.notnull()

# Generamos el objeto matc sin missings values
data_nuevo['property_title'] = property_match_result[property_match_notnull].apply(lambda x: x if x is np.NaN else x.group(0))

In [107]:
data_nuevo.loc[:,['title','property_title']]

,title,property_title
0,2 AMB TIPO CASA SIN EXPENSAS EN PB,CASA
1,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,NaN
2,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,NaN
3,PH 3 amb. cfte. reciclado,NaN
4,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,NaN
...,...,...
121215,Torre Forum Alcorta- Impecable 3 ambientes,NaN
121216,Ruca Inmuebles | Venta | Lomas de San Isidro |...,NaN
121217,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,NaN
121218,2 amb. C/ dep. de servicio al contrafrente| Re...,NaN


In [ ]:
data_nuevo.property_title.notnull().sum()

# 6. Resultados Regex de las columnas "description" y "title"


### Regex description

* M2 = rescatamos 30.041
* Rooms = rescatamos  39.761
* Dólares = rescatamos 3.924 
* Tipo propiedad = 36.807

### Regex title 
* M2 = rescatamos 6222
* Rooms = rescatamos  39.761
* Dólares = rescatamos 2.534
* Tipo  de propiedad  =36. 807


# 7. Visualizamos y agregamos los datos obtenidos en description y title 

### 7.1 Rellenamos valores nulos de "rooms" con "rooms_desc"

In [108]:
# Creamos las series de rooms y rooms_desc
rooms_desc_serie = data_nuevo.rooms_desc
rooms_serie = data_nuevo.rooms

# Convertimos rooms_desc a float
data_nuevo["rooms_desc"] = rooms_desc_serie.astype(float)

# Reemplazamos los missings values que esten en rooms por los valores que tenga rooms_desc
data_nuevo['rooms'] = rooms_serie.fillna(rooms_desc_serie)

In [109]:
data_nuevo.loc[:,['rooms','rooms_desc']]

,rooms,rooms_desc
0,2,2.0
1,NaN,NaN
2,2,2.0
3,3,3.0
4,NaN,NaN
...,...,...
121215,NaN,NaN
121216,NaN,NaN
121217,NaN,NaN
121218,2,2.0


In [110]:
data_nuevo.rooms.notnull().sum()

67594

### 7.2 Rellenamos valores nulos de surface_total_in_m2 con m2_desc

In [ ]:
# Creramos la serie de surface_total_in_m2 y m2_desc
#surface_total_in_m2_serie = data_nuevo.surface_total_in_m2
#m2_desc_serie_str = data_nuevo.m2_desc.astype(str)

# 1-Reemplazar valores con caracteres especiales delante de los montos iguales o superiores a mil
# Creo el patron nuevo
#pattern_suffix = "(?P<numbers>^\d?\d?\d?.?\d?\d?\d)" 

# Compilo el nuevo patron
#pattern_suffix_regex = re.compile(pattern_suffix)

# Generamos el nuevo objeto match
#m2_new_match = m2_desc_serie_str.apply(lambda x: pattern_suffix_regex.search(x))

# Creamos una boolean mask sin missings values
#m2_new_match_notnull = m2_new_match.notnull()

# Generamos el nuevo objeto match sin missings values
#data_nuevo['m2_desc'] = m2_new_match[m2_new_match_notnull].apply(lambda x: x if x is np.NaN else x.group('numbers'))

In [ ]:
# 1-Reemplazar valores con string delante de los montos inferioes a mil
# Creo el patron nuevo
#pattern_scar = '[A-Z]'
 
# Compilo el nuevo patron
#pattern_scar_regex = re.compile(pattern_scar)

# Reemplazo los caracteres especiales
#cadena_reemplazo = ''

# Generamos el nuevo objeto match de reemplazo
#data_nuevo['m2_desc'] = m2_desc_serie_str.apply(lambda x: pattern_scar_regex.sub(cadena_reemplazo,x))

In [ ]:
# 2-Reemplazar valores con caracteres especiales de los montos inferioes a mil
# Creo el patron nuevo
#pattern_scar = '\D[^,]*'

# Compilo el nuevo patron
#pattern_scar_regex = re.compile(pattern_scar)

# Reemplazo los caracteres especiales
#cadena_reemplazo = ''

# Generamos el nuevo objeto match de reemplazo
#data_nuevo['m2_desc'] = m2_desc_serie_str.apply(lambda x: pattern_scar_regex.sub(cadena_reemplazo,x))

In [ ]:
# verificamos que los valores obtenidos tengan un formato float64 limpio 

#data_nuevo['m2_desc'].unique().tolist()

In [ ]:
# Convertimos la columna m2_desc a float
#data_nuevo['m2_desc'] = pd.to_numeric(data_nuevo['m2_desc'],errors='coerce')

# Creamos la boolean mask con valores mayores o iguales a 10 m2 
#more_10m2_mask = data_nuevo['m2_desc'] >= 10

# Colocamos la boolean mask dentro de la serie de m2_desc
#data_nuevo['m2_desc'] = data_nuevo.m2_desc.loc[more_10m2_mask]

In [ ]:
#data_nuevo.loc[:,['surface_total_in_m2','m2_desc','description']]

In [ ]:
# Reemplazamos los missings values que esten en surface_total_in_m2 por los valores que tenga m2_desc
#data_nuevo['surface_total_in_m2'] = surface_total_in_m2_serie.fillna(data_nuevo.m2_desc)

In [ ]:
#data_nuevo['surface_total_in_m2'].notnull().sum()

Conclusion: La información extraida de la columna description no nos aporta un volumen significativo de datos que se puedan recuperar para la columna surface_total_in_m2

### 7.3 Rellenamos valores nulos de "price_aprox_usd" con la columna "dollar_desc"


In [111]:
# Evaluamos coherencia y correcta aplicacion del match
data_nuevo.loc[0:5,['description','dollar_desc',"price_title"]]

,description,dollar_desc,price_title
0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",NaN,NaN
1,Venta de departamento en décimo piso al frente...,U$D 20.000,NaN
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,NaN,NaN
3,PH 3 ambientes con patio. Hay 3 deptos en lote...,NaN,NaN
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,NaN,NaN
5,"Casa en el perímetro del barrio 338, ubicada e...",NaN,NaN


In [112]:
#vamos a ver si hay duplicados entre la columnas "dollar_desc", "price_title"


duplicated_mask_dollar = data_nuevo.duplicated(subset = [ "dollar_desc", "price_title"], keep = "first")

print(any(duplicated_mask_dollar))

duplicated_dollar = data_nuevo.loc[duplicated_mask_dollar]
duplicated_dollar.sort_values(by=["dollar_desc", "price_title"]).head(5)

True


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,Provincia,Barrio,m2_desc,rooms_desc,dollar_desc,property_desc,rooms_title,price_title,m2_title,property_title
92149,92149,sell,house,Punilla,|Argentina|Córdoba|Punilla|,Argentina,Córdoba,3839996.0,NaN,NaN,...,Córdoba,Punilla,NaN,NaN,U$D 100.000,CASA,NaN,NaN,NaN,CASA
119300,119300,sell,PH,Villa Ballester,|Argentina|Bs.As. G.B.A. Zona Norte|General Sa...,Argentina,Bs.As. G.B.A. Zona Norte,3427467.0,"-34.5454848068,-58.5673549524",-34.545485,...,Bs.As. G.B.A. Zona Norte,General San Martín,NaN,NaN,U$D 100.000,CASA,NaN,NaN,NaN,NaN
61368,61368,sell,apartment,San Martín de los Andes,|Argentina|Neuquén|San Martín de los Andes|,Argentina,Neuquén,3836951.0,"-40.1539514,-71.3449129",-40.153951,...,Neuquén,San Martín de los Andes,53.74,NaN,U$D 105.000,NaN,NaN,NaN,NaN,NaN
89633,89633,sell,apartment,Villa Ballester,|Argentina|Bs.As. G.B.A. Zona Norte|General Sa...,Argentina,Bs.As. G.B.A. Zona Norte,3427467.0,"-34.5443544,-58.5686386",-34.544354,...,Bs.As. G.B.A. Zona Norte,General San Martín,38,3.0,U$D 108.000,NaN,NaN,NaN,NaN,NaN
39745,39745,sell,apartment,San Luis,|Argentina|San Luis|San Luis|,Argentina,San Luis,3837056.0,"-33.2948254,-66.3360086",-33.294825,...,San Luis,San Luis,NaN,NaN,U$D 110.000,NaN,NaN,NaN,NaN,NaN


In [113]:
#una vez detectados que hay duplicados vamos a eliminarlos quedandonos con los que aparezcan en description

data_dollar_nodup = duplicated_dollar.drop_duplicates(subset = ["dollar_desc", "price_title"], keep = "first")
data_nuevo.dollar_desc.notnull().sum()

3924

In [114]:
#Con lo cual vamos a rellenar la columna de price price_aprox_usd con la informacion extraida en description sobre los precios en usd de los inmuebles
# Creamos las series de price_aprox_usd y dollar_desc
price_aprox_usd_serie = data_nuevo.price_aprox_usd
dollar_desc_serie = data_nuevo.dollar_desc

# Convertimos rooms_desc a float
#data_nuevo["rooms_desc"] = rooms_desc_serie.astype(float)

# Reemplazamos los missings values que esten en rooms por los valores que tenga rooms_desc
#data_nuevo['price_aprox_usd'] = price_aprox_usd_serie.fillna(dollar_desc_serie)

In [115]:
#vamos a ver cuantos valores salavamos comparando la situacion el numero anterio de not nulls

data_nuevo.price_aprox_usd.notnull().sum()

100702

# 8. Conclusiones sobre los Regex 


### 8.1 Conclusión "title" 

Luego de analizar dicha columna determinamos que el aporte de la columna era insignificante y no tan fiable, razon por la cual nos centramos en utilizar solamente la columna "description"





### 8.2 Conclusión "description"

Luego del proceso de extraccion de datos de la columna description aplicando regex aplicamos un fillna sobre las columnas originales y tuvimos los siguientes resultados:

+ **Rooms**: Antes de extraer datos de cantidad de rooms de la columna description, la cantidad de valores no nulos era 47.390, y luego de reemplazar los NaNs con los valores de rooms_desc, la cantidad de valores no nulos asciende a 67.594 , lo que significa que los esfuerzos de extraccion aportaron un aumento de datos del 43% (y en cantidad 20.204).

+ **M2**: Antes de extraer datos de cantidad de m2 de la columna description, la cantidad de valores no nulos era de 81.892 y luego de reemplazar los NaNs con los valores de m2_desc, la cantidad de valores no nulos asciende a 83.491, lo que significa que los esfuerzos de extraccion de informacion aportaron un aumento poco significativo de datos del 1.95% (y en cantidad 1.599).  A su vez no podemos discernir si la superficie que obtenemos es cubierta o total.

+ **Precio en Dólares**: Antes de extraer informacion de precios en dolares de la columna description, la cantidad de valores no nulos era de 100.810 y luego de reemplazar los NaNs con los valores de dollar_desc, la cantidad de valores no nulos asciende a 100.939, lo que aporta un porcentaje poco significativo del 0.13% (en cantidad 129 datos)

#### En relacion a los datos obtenidos decidimos NO utilizar la variables dolar_desc  y M2_desc, por otro lado si vamos a considerar usar la variable rooms_desc como un potencial dato para el posterior analisis de decisiones a tomar. 

# 9. Nuevo Dataset limpio 

In [116]:
data_nuevo.surface_total_in_m2.sort_values()

50696     10.0
47471     10.0
48479     10.0
14887     10.0
14873     10.0
          ... 
121155     NaN
121156     NaN
121207     NaN
121208     NaN
121210     NaN
Name: surface_total_in_m2, Length: 121220, dtype: float64

### 9.1 Generamos un nuevo dataset en base a la columna que vamos a usar para establecer las relaciones 

In [117]:
data_nuevo.columns


Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail', 'Provincia', 'Barrio', 'm2_desc', 'rooms_desc',
       'dollar_desc', 'property_desc', 'rooms_title', 'price_title',
       'm2_title', 'property_title'],
      dtype='object')

In [ ]:
# Vamos a sacar las columnas que no nos sirven para quedarnos con un dataframe reducido y limpio


#data_nuevo.drop(columns=['place_with_parent_names','geonames_id', 'lat-lon','price_per_m2',"description","title",'price_usd_per_m2','lat', 'lon','rooms_desc','dollar_desc', 'm2_desc','property_desc', 'rooms_title', 'price_title','m2_title', 'property_title'],inplace=True)

In [118]:
data_nuevo


,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,Provincia,Barrio,m2_desc,rooms_desc,dollar_desc,property_desc,rooms_title,price_title,m2_title,property_title
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,Capital Federal,Mataderos,NaN,2.0,NaN,CASA,2,NaN,NaN,CASA
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,Bs.As. G.B.A. Zona Sur,La Plata,NaN,NaN,U$D 20.000,NaN,NaN,NaN,NaN,NaN
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,Capital Federal,Mataderos,NaN,2.0,NaN,NaN,2,NaN,NaN,NaN
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,Capital Federal,Liniers,NaN,3.0,NaN,NaN,3,NaN,NaN,NaN
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,Buenos Aires Costa Atlántica,Mar del Plata,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,Capital Federal,Belgrano,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN
121216,121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,...,Bs.As. G.B.A. Zona Norte,San Isidro,572,NaN,NaN,CASA,NaN,NaN,NaN,NaN
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,Capital Federal,Villa Urquiza,"43,00",NaN,USD 20.000,NaN,NaN,NaN,NaN,NaN
121218,121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,...,Buenos Aires Costa Atlántica,Mar del Plata,NaN,2.0,NaN,NaN,2,NaN,NaN,NaN


# 9.2 Exportamos el dataframe 

In [119]:
#exportamos el archivo para trabajarlo en la notebook 3 

ubicacion = r"C:\Users\usuario\Desktop\RELIQUIDACIONES\data_limpio.csv"



In [120]:
data_nuevo.to_csv(ubicacion)

In [121]:
data_nuevo.surface_total_in_m2.isnull().sum()

27344

In [122]:
data_nuevo.shape[0]

121220